<a href="https://colab.research.google.com/github/cff100/MecFLu2/blob/main/2023_combined_all_pressure_levels_ordered.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cdsapi
!pip install cartopy
!pip install netcdf4
!pip install openpyxl


In [ ]:
import math
import cdsapi
import xarray as xr
import pandas as pd
from datetime import datetime, timedelta

# Função para calcular a altura a partir da pressão
def calcular_altura(PA, k, P_h, g):
    altura = -math.log(P_h / PA) / (k * g)
    return altura

# Função para converter horário de Brasília para UTC
def brasilia_to_utc(hour_brasilia):
    brasilia_time = datetime.strptime(hour_brasilia, '%H:%M')
    utc_time = brasilia_time - timedelta(hours=3)
    return utc_time.strftime('%H:%M')

# Função para converter horário UTC para horário de Brasília
def utc_to_brasilia(hour_utc):
    utc_time = datetime.strptime(hour_utc, '%H:%M')
    brasilia_time = utc_time + timedelta(hours=3)
    return brasilia_time.strftime('%H:%M')

# Parâmetros fornecidos pelo usuário
PA = 101.325  # Pressão atmosférica ao nível do mar em kPa
k = 1.15e-4   # Constante específica
g = 9.81      # Aceleração gravitacional em m/s²

# Inicializa a API do CDS
c = cdsapi.Client(url='https://cds.climate.copernicus.eu/api/v2', key='331408:d8c35ac5-fae8-4f2e-90fe-b448d76e3140')

# Níveis de pressão em hPa
pressure_levels = [82.5, 85.0, 87.5, 90.0, 92.5, 95.0, 97.5]

# Coordenadas das plataformas
platforms = {
    'NAMORADO 2 (PNA-2)': (-22.45073, -40.41175),
    'PETROBRAS 26 (P-26)': (-22.4684, -40.02869),
    'PETROBRAS 32 (P-32)': (-22.2051, -40.1431),
    'PETROBRAS 37 (P-37)': (-22.4868, -40.09779),
    'PETROBRAS IX': (-22.57358, -40.82192),
    'PETROBRAS XIX': (-22.3927, -40.05438),
    'PETROBRAS XXXIII': (-22.37, -40.0267),
    'VERMELHO 1 (PVM-1)': (-22.16065, -40.27872),
    'VERMELHO 2 (PVM-2)': (-22.17535, -40.29147),
}

# Hora de Brasília a ser convertida
brasilia_hour = '03:00'
utc_hour = brasilia_to_utc(brasilia_hour)

# Cria um DataFrame vazio para armazenar os resultados combinados de todas as pressões
combined_df = pd.DataFrame()

# Loop sobre cada nível de pressão
for level in pressure_levels:
    # Nome do arquivo NetCDF para o nível de pressão atual com o horário de Brasília
    nc_filename = f'Era5_2023_{int(level*10)}HPA_CAMPOS_{brasilia_hour}.nc'

    # Requisição dos dados
    c.retrieve(
        'reanalysis-era5-pressure-levels',
        {
            'product_type': 'reanalysis',
            'format': 'netcdf',
            'variable': ['u_component_of_wind', 'v_component_of_wind'],
            'pressure_level': f'{int(level*10)}',  # Converte o nível para hPa
            'year': '2023',
            'month': [str(i).zfill(2) for i in range(1, 13)],
            'day': [str(i).zfill(2) for i in range(1, 32)],
            'time': [utc_hour],
            'area': [-27, -45, -20, -35],  # Coordenadas da Bacia de Campos
        },
        nc_filename)

    # Carrega o arquivo NetCDF usando xarray
    ds = xr.open_dataset(nc_filename)

    # Calcula a velocidade do vento (wspd)
    wspd = (ds['u']**2 + ds['v']**2)**0.5

    # Calcula a média da velocidade do vento por estação do ano
    wspd_mean_season = wspd.groupby('time.season').mean(dim=['time'])

    # Calcula a altura correspondente ao nível de pressão atual
    altura = calcular_altura(PA, k, level, g)

    # Cria um DataFrame para armazenar os resultados específicos para o nível de pressão atual
    result_df = pd.DataFrame(index=platforms.keys())

    # Loop sobre as estações do ano
    for season in ['DJF', 'MAM', 'JJA', 'SON']:
        for platform, (lat, lon) in platforms.items():
            # Interpola o valor da média da velocidade do vento
            value = wspd_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()
            result_df.loc[platform, f'{season}'] = value

    # Adiciona as colunas de pressão e altura
    result_df['Pressure_Level'] = f'{int(level*10)} HPA'
    result_df['Altitude (m)'] = altura
    result_df['Brasilia_Time'] = brasilia_hour
    result_df['UTC_Time'] = utc_hour

    # Adiciona o DataFrame do nível de pressão atual ao DataFrame combinado
    combined_df = pd.concat([combined_df, result_df])

    # Fecha o arquivo NetCDF
    ds.close()



In [ ]:
# Hora de Brasília a ser convertida
brasilia_hour = '03:00'
utc_hour = brasilia_to_utc(brasilia_hour)

# Cria um DataFrame vazio para armazenar os resultados combinados de todas as pressões
combined_df = pd.DataFrame()

# Loop sobre cada nível de pressão
for level in pressure_levels:
    # Nome do arquivo NetCDF para o nível de pressão atual com o horário de Brasília
    nc_filename = f'Era5_2023_{int(level*10)}HPA_CAMPOS_{brasilia_hour}.nc'

    # Requisição dos dados
    c.retrieve(
        'reanalysis-era5-pressure-levels',
        {
            'product_type': 'reanalysis',
            'format': 'netcdf',
            'variable': ['u_component_of_wind', 'v_component_of_wind'],
            'pressure_level': f'{int(level*10)}',  # Converte o nível para hPa
            'year': '2023',
            'month': [str(i).zfill(2) for i in range(1, 13)],
            'day': [str(i).zfill(2) for i in range(1, 32)],
            'time': [utc_hour],
            'area': [-27, -45, -20, -35],  # Coordenadas da Bacia de Campos
        },
        nc_filename)

    # Carrega o arquivo NetCDF usando xarray
    ds = xr.open_dataset(nc_filename)

    # Calcula a velocidade do vento (wspd)
    wspd = (ds['u']**2 + ds['v']**2)**0.5

    # Calcula a média da velocidade do vento por estação do ano
    wspd_mean_season = wspd.groupby('time.season').mean(dim=['time'])

    # Calcula a altura correspondente ao nível de pressão atual
    altura = calcular_altura(PA, k, level, g)

    # Cria um DataFrame para armazenar os resultados específicos para o nível de pressão atual
    result_df = pd.DataFrame(index=platforms.keys())

    # Loop sobre as estações do ano
    for season in ['DJF', 'MAM', 'JJA', 'SON']:
        for platform, (lat, lon) in platforms.items():
            # Interpola o valor da média da velocidade do vento
            value = wspd_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()
            result_df.loc[platform, f'{season}'] = value

    # Adiciona as colunas de pressão e altura
    result_df['Pressure_Level'] = f'{int(level*10)} HPA'
    result_df['Altitude (m)'] = altura
    result_df['Brasilia_Time'] = brasilia_hour
    result_df['UTC_Time'] = utc_hour

    # Adiciona o DataFrame do nível de pressão atual ao DataFrame combinado
    combined_df = pd.concat([combined_df, result_df])

    # Fecha o arquivo NetCDF
    ds.close()

# Ajusta a ordem das colunas
combined_df = combined_df[['Pressure_Level', 'Altitude (m)', 'Brasilia_Time', 'UTC_Time', 'DJF', 'MAM', 'JJA', 'SON']]

# Salva o DataFrame combinado como CSV
combined_df.to_csv(f'2023_combined_pressure_levels_with_altitude_{brasilia_hour}.csv', index=True)

# Salva o DataFrame combinado como Excel
combined_df.to_excel(f'2023_combined_pressure_levels_with_altitude_{brasilia_hour}.xlsx', index=True)

2024-08-29 20:32:43,088 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
INFO:cdsapi:Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-08-29 20:32:43,092 WARNING MOVE TO

Exception: no data is available within your requested subset. Request returned no data.

In [5]:
# Hora de Brasília a ser convertida
brasilia_hour = '09:00'
utc_hour = brasilia_to_utc(brasilia_hour)

# Cria um DataFrame vazio para armazenar os resultados combinados de todas as pressões
combined_df = pd.DataFrame()

# Loop sobre cada nível de pressão
for level in pressure_levels:
    # Nome do arquivo NetCDF para o nível de pressão atual com o horário de Brasília
    nc_filename = f'Era5_2023_{int(level*10)}HPA_CAMPOS_{brasilia_hour}.nc'

    # Requisição dos dados
    c.retrieve(
        'reanalysis-era5-pressure-levels',
        {
            'product_type': 'reanalysis',
            'format': 'netcdf',
            'variable': ['u_component_of_wind', 'v_component_of_wind'],
            'pressure_level': f'{int(level*10)}',  # Converte o nível para hPa
            'year': '2023',
            'month': [str(i).zfill(2) for i in range(1, 13)],
            'day': [str(i).zfill(2) for i in range(1, 32)],
            'time': [utc_hour],
            'area': [-27, -45, -20, -35],  # Coordenadas da Bacia de Campos
        },
        nc_filename)

    # Carrega o arquivo NetCDF usando xarray
    ds = xr.open_dataset(nc_filename)

    # Calcula a velocidade do vento (wspd)
    wspd = (ds['u']**2 + ds['v']**2)**0.5

    # Calcula a média da velocidade do vento por estação do ano
    wspd_mean_season = wspd.groupby('time.season').mean(dim=['time'])

    # Calcula a altura correspondente ao nível de pressão atual
    altura = calcular_altura(PA, k, level, g)

    # Cria um DataFrame para armazenar os resultados específicos para o nível de pressão atual
    result_df = pd.DataFrame(index=platforms.keys())

    # Loop sobre as estações do ano
    for season in ['DJF', 'MAM', 'JJA', 'SON']:
        for platform, (lat, lon) in platforms.items():
            # Interpola o valor da média da velocidade do vento
            value = wspd_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()
            result_df.loc[platform, f'{season}'] = value

    # Adiciona as colunas de pressão e altura
    result_df['Pressure_Level'] = f'{int(level*10)} HPA'
    result_df['Altitude (m)'] = altura
    result_df['Brasilia_Time'] = brasilia_hour
    result_df['UTC_Time'] = utc_hour

    # Adiciona o DataFrame do nível de pressão atual ao DataFrame combinado
    combined_df = pd.concat([combined_df, result_df])

    # Fecha o arquivo NetCDF
    ds.close()

# Ajusta a ordem das colunas
combined_df = combined_df[['Pressure_Level', 'Altitude (m)', 'Brasilia_Time', 'UTC_Time', 'DJF', 'MAM', 'JJA', 'SON']]

# Salva o DataFrame combinado como CSV
combined_df.to_csv(f'2023_combined_pressure_levels_with_altitude_{brasilia_hour}.csv', index=True)

# Salva o DataFrame combinado como Excel
combined_df.to_excel(f'2023_combined_pressure_levels_with_altitude_{brasilia_hour}.xlsx', index=True)


2024-08-29 20:35:12,569 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
INFO:cdsapi:Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-08-29 20:35:12,574 WARNING MOVE TO

Exception: no data is available within your requested subset. Request returned no data.

In [6]:
# Ajusta a ordem das colunas
combined_df = combined_df[['Pressure_Level', 'Altitude (m)', 'Brasilia_Time', 'UTC_Time', 'DJF', 'MAM', 'JJA', 'SON']]

# Salva o DataFrame combinado como CSV
combined_df.to_csv(f'2023_combined_pressure_levels_with_altitude_{brasilia_hour}.csv', index=True)

# Salva o DataFrame combinado como Excel
combined_df.to_excel(f'2023_combined_pressure_levels_with_altitude_{brasilia_hour}.xlsx', index=True)


In [7]:
# Hora de Brasília a ser convertida
brasilia_hour = '15:00'
utc_hour = brasilia_to_utc(brasilia_hour)

# Cria um DataFrame vazio para armazenar os resultados combinados de todas as pressões
combined_df = pd.DataFrame()

# Loop sobre cada nível de pressão
for level in pressure_levels:
    # Nome do arquivo NetCDF para o nível de pressão atual com o horário de Brasília
    nc_filename = f'Era5_2023_{int(level*10)}HPA_CAMPOS_{brasilia_hour}.nc'

    # Requisição dos dados
    c.retrieve(
        'reanalysis-era5-pressure-levels',
        {
            'product_type': 'reanalysis',
            'format': 'netcdf',
            'variable': ['u_component_of_wind', 'v_component_of_wind'],
            'pressure_level': f'{int(level*10)}',  # Converte o nível para hPa
            'year': '2023',
            'month': [str(i).zfill(2) for i in range(1, 13)],
            'day': [str(i).zfill(2) for i in range(1, 32)],
            'time': [utc_hour],
            'area': [-27, -45, -20, -35],  # Coordenadas da Bacia de Campos
        },
        nc_filename)

    # Carrega o arquivo NetCDF usando xarray
    ds = xr.open_dataset(nc_filename)

    # Calcula a velocidade do vento (wspd)
    wspd = (ds['u']**2 + ds['v']**2)**0.5

    # Calcula a média da velocidade do vento por estação do ano
    wspd_mean_season = wspd.groupby('time.season').mean(dim=['time'])

    # Calcula a altura correspondente ao nível de pressão atual
    altura = calcular_altura(PA, k, level, g)

    # Cria um DataFrame para armazenar os resultados específicos para o nível de pressão atual
    result_df = pd.DataFrame(index=platforms.keys())

    # Loop sobre as estações do ano
    for season in ['DJF', 'MAM', 'JJA', 'SON']:
        for platform, (lat, lon) in platforms.items():
            # Interpola o valor da média da velocidade do vento
            value = wspd_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()
            result_df.loc[platform, f'{season}'] = value

    # Adiciona as colunas de pressão e altura
    result_df['Pressure_Level'] = f'{int(level*10)} HPA'
    result_df['Altitude (m)'] = altura
    result_df['Brasilia_Time'] = brasilia_hour
    result_df['UTC_Time'] = utc_hour

    # Adiciona o DataFrame do nível de pressão atual ao DataFrame combinado
    combined_df = pd.concat([combined_df, result_df])

    # Fecha o arquivo NetCDF
    ds.close()



2024-08-29 21:51:03,650 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
INFO:cdsapi:Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-08-29 21:51:03,654 WARNING MOVE TO

Exception: no data is available within your requested subset. Request returned no data.

In [8]:
# Ajusta a ordem das colunas
combined_df = combined_df[['Pressure_Level', 'Altitude (m)', 'Brasilia_Time', 'UTC_Time', 'DJF', 'MAM', 'JJA', 'SON']]

# Salva o DataFrame combinado como CSV
combined_df.to_csv(f'2023_combined_pressure_levels_with_altitude_{brasilia_hour}.csv', index=True)

# Salva o DataFrame combinado como Excel
combined_df.to_excel(f'2023_combined_pressure_levels_with_altitude_{brasilia_hour}.xlsx', index=True)


In [9]:
# Hora de Brasília a ser convertida
brasilia_hour = '21:00'
utc_hour = brasilia_to_utc(brasilia_hour)

# Cria um DataFrame vazio para armazenar os resultados combinados de todas as pressões
combined_df = pd.DataFrame()

# Loop sobre cada nível de pressão
for level in pressure_levels:
    # Nome do arquivo NetCDF para o nível de pressão atual com o horário de Brasília
    nc_filename = f'Era5_2023_{int(level*10)}HPA_CAMPOS_{brasilia_hour}.nc'

    # Requisição dos dados
    c.retrieve(
        'reanalysis-era5-pressure-levels',
        {
            'product_type': 'reanalysis',
            'format': 'netcdf',
            'variable': ['u_component_of_wind', 'v_component_of_wind'],
            'pressure_level': f'{int(level*10)}',  # Converte o nível para hPa
            'year': '2023',
            'month': [str(i).zfill(2) for i in range(1, 13)],
            'day': [str(i).zfill(2) for i in range(1, 32)],
            'time': [utc_hour],
            'area': [-27, -45, -20, -35],  # Coordenadas da Bacia de Campos
        },
        nc_filename)

    # Carrega o arquivo NetCDF usando xarray
    ds = xr.open_dataset(nc_filename)

    # Calcula a velocidade do vento (wspd)
    wspd = (ds['u']**2 + ds['v']**2)**0.5

    # Calcula a média da velocidade do vento por estação do ano
    wspd_mean_season = wspd.groupby('time.season').mean(dim=['time'])

    # Calcula a altura correspondente ao nível de pressão atual
    altura = calcular_altura(PA, k, level, g)

    # Cria um DataFrame para armazenar os resultados específicos para o nível de pressão atual
    result_df = pd.DataFrame(index=platforms.keys())

    # Loop sobre as estações do ano
    for season in ['DJF', 'MAM', 'JJA', 'SON']:
        for platform, (lat, lon) in platforms.items():
            # Interpola o valor da média da velocidade do vento
            value = wspd_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()
            result_df.loc[platform, f'{season}'] = value

    # Adiciona as colunas de pressão e altura
    result_df['Pressure_Level'] = f'{int(level*10)} HPA'
    result_df['Altitude (m)'] = altura
    result_df['Brasilia_Time'] = brasilia_hour
    result_df['UTC_Time'] = utc_hour

    # Adiciona o DataFrame do nível de pressão atual ao DataFrame combinado
    combined_df = pd.concat([combined_df, result_df])

    # Fecha o arquivo NetCDF
    ds.close()



2024-08-29 22:20:58,854 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
INFO:cdsapi:Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-08-29 22:20:58,858 WARNING MOVE TO

Exception: no data is available within your requested subset. Request returned no data.

In [10]:
# Ajusta a ordem das colunas
combined_df = combined_df[['Pressure_Level', 'Altitude (m)', 'Brasilia_Time', 'UTC_Time', 'DJF', 'MAM', 'JJA', 'SON']]

# Salva o DataFrame combinado como CSV
combined_df.to_csv(f'2023_combined_pressure_levels_with_altitude_{brasilia_hour}.csv', index=True)

# Salva o DataFrame combinado como Excel
combined_df.to_excel(f'2023_combined_pressure_levels_with_altitude_{brasilia_hour}.xlsx', index=True)


In [19]:
import pandas as pd
import glob

# Lista de horários na ordem desejada
horarios_brasilia = ['03:00', '09:00', '15:00', '21:00']

# Lista para armazenar os DataFrames na ordem correta
dataframes = []

# Loop sobre cada horário e carregar o DataFrame correspondente
for horario in horarios_brasilia:
    # Buscar o arquivo CSV correspondente ao horário atual
    csv_file = glob.glob(f'2023_combined_pressure_levels_with_altitude_{horario}.csv')

    if csv_file:
        # Carregar o DataFrame e adicionar à lista
        df = pd.read_csv(csv_file[0])
        dataframes.append(df)

# Concatenar os DataFrames na ordem correta
combined_df = pd.concat(dataframes, ignore_index=True)

# Renomeando a coluna
#combined_df.rename(columns={'Unnamed: 0': 'Plataformas'}, inplace=True)
# Renomeando as colunas
combined_df.rename(columns={
    'Unnamed: 0': 'Plataforms',
    'Altitude (m)': 'Height_(m)'
}, inplace=True)

# Salvar o DataFrame combinado como CSV
combined_df.to_csv('2023_combined_all_pressure_levels_ordered.csv', index=False)

# Salvar o DataFrame combinado como Excel
combined_df.to_excel('2023_combined_all_pressure_levels_ordered.xlsx', index=False)
